In [1]:
import pandas as pd
import numpy as np

Funcion para quitar dejar en nuestro archivo solo los valores sin / y sin ; de nuestro clasificador

In [3]:
def filtro_df(df):
    #filtramos las barras
    filtro_barra = ~df['CLIN_SIG'].str.contains('/')
    df= df.loc[filtro_barra]
    #filtramos los ;
    filtro_puntocoma = ~df['CLIN_SIG'].str.contains(';')
    df= df.loc[filtro_puntocoma]
    #reseteamos el indice
    df.reset_index()
    
    return df

Funcion para coger "x" lineas de cada categoría para el df final

In [1]:
def sample_rows_per_category(dataframe, category_column, n_rows):
    # Group by the category column and sample n_rows from each category
    return dataframe.groupby(category_column).apply(lambda x: x.sample(n=n_rows)).reset_index(drop=True)

Cargamos el archivo vcf

In [5]:
colnames = ['Uploaded_variation','Location','Allele','Gene','Feature','Feature_type','Consequence','cDNA_position'
,'CDS_position','Protein_position','Amino_acids','Codons','Existing_variation','IMPACT','DISTANCE','STRAND','FLAGS','BIOTYPE',
            'CANONICAL','REFSEQ_MATCH','SOURCE','REFSEQ_OFFSET','GIVEN_REF','USED_REF','BAM_EDIT','SIFT','PolyPhen','AF',
            'gnomADg_AF','gnomADg_AFR_AF','gnomADg_AMI_AF','gnomADg_AMR_AF','gnomADg_ASJ_AF','gnomADg_EAS_AF',
            'gnomADg_FIN_AF','gnomADg_MID_AF','gnomADg_NFE_AF','gnomADg_OTH_AF','gnomADg_SAS_AF','CLIN_SIG','SOMATIC','PHENO']

In [6]:
df = pd.read_csv(f'clin_an_2_rs.vcf', sep='\t', comment='#', names=colnames, header=None, index_col=False)

In [7]:
palabras_clave = ['likely_benign', 'benign', 'pathogenic', 'likely_pathogenic']

Creamos un filtro que nos coge solo nuestros clasificadores clave y que no tienen una , en el texto y lo aplicamos

In [8]:
filtro_clin_sig = df['CLIN_SIG'].str.contains('|'.join([rf'\b{palabra}\b' for palabra in palabras_clave]), regex=True)& (~df['CLIN_SIG'].str.contains(','))

In [9]:
df_filtrado= df.loc[filtro_clin_sig]

In [10]:
df_filtrado = filtro_df(df_filtrado)

Creams ua copia de nuestro df para dejar intacto el original

In [13]:
df_filtrado_canonico = df_filtrado.copy()

Filtramos por la categoría de frecuencia alelica que no sea un "-"

In [29]:
df_filtrado_canonico = df_filtrado_canonico[df_filtrado_canonico['gnomADg_AF']!='-']

In [20]:
df_filtrado_canonico['CLIN_SIG'].value_counts()

likely_benign        3590254
benign               1785174
pathogenic            700185
likely_pathogenic     349096
Name: CLIN_SIG, dtype: int64

Quitamos las columnas sobrantes 

In [15]:
columns_to_drop = ['Uploaded_variation','Location','cDNA_position','CDS_position','Protein_position','Gene','Feature','Feature_type',
                   'Existing_variation','DISTANCE','STRAND','CANONICAL','REFSEQ_MATCH',
                   'SOURCE', 'REFSEQ_OFFSET','GIVEN_REF','BAM_EDIT','PolyPhen','SIFT','AF']
# columns_to_drop = ['Uploaded_variation','Location','cDNA_position','CDS_position','Protein_position','Gene','Feature','Feature_type',
#                    'Existing_variation','DISTANCE','STRAND','CANONICAL','REFSEQ_MATCH',
#                    'SOURCE', 'REFSEQ_OFFSET','GIVEN_REF','BAM_EDIT','SIFT','AF']

In [16]:
df_filtrado_canonico = df_filtrado_canonico.drop(columns=columns_to_drop)

Convertimos las columnas con datos numericos en numeros, primero remplazamos los - de los valores faltantes por nan y luego las convertimos a número.

In [17]:
columns_to_convert  = ['gnomADg_AF','gnomADg_AFR_AF','gnomADg_AMI_AF','gnomADg_AMR_AF','gnomADg_ASJ_AF','gnomADg_EAS_AF',
            'gnomADg_FIN_AF','gnomADg_MID_AF','gnomADg_NFE_AF','gnomADg_OTH_AF','gnomADg_SAS_AF']

In [18]:
 df_filtrado_canonico[columns_to_convert] = df_filtrado_canonico[columns_to_convert].replace('-', np.nan)

In [19]:
df_filtrado_canonico[columns_to_convert] = df_filtrado_canonico[columns_to_convert].apply(pd.to_numeric)

Para los valores faltantes calculamos la media de los datos de freccuencias de sus filas

In [31]:
for c in columns_to_convert:
    columnas_para_media = [col for col in columns_to_convert if col not in [c]]

    media = df_filtrado_canonico[columnas_para_media].mean(axis=1)  # Calcula la media de las columnas excepto 'gnomADg_AF'

    df_filtrado_canonico[c].fillna(media, inplace=True)

Muestreamos y mezclamos los datos del df

In [36]:

sampled_df = sample_rows_per_category(df_filtrado_canonico, 'CLIN_SIG', 300000)

In [37]:
shuffled_df = sampled_df.sample(frac=1).reset_index(drop=True)

In [39]:
shuffled_df.to_csv('df_filtrado_22.csv', sep = '\t',index=False)